### Importing all the necessary data 

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow	import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

### Data Loading and observing

In [116]:
dataset = pd.read_csv('admissions_data.csv')
dataset.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
5,6,330,115,5,4.5,3.0,9.34,1,0.90
6,7,321,109,3,3.0,4.0,8.20,1,0.75
7,8,308,101,2,3.0,4.0,7.90,0,0.68
8,9,302,102,1,2.0,1.5,8.00,0,0.50
9,10,323,108,3,3.5,3.0,8.60,0,0.45


In [117]:
dataset.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [118]:
# Splitting the dataset into labels and features:
labels = dataset.iloc[:, -1]
features = dataset.iloc[:, 1:-1]

### Data preprocessing

In [119]:
# Splitting your data into training set and test sets:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.20, random_state = 23)


# Normalizing the numeric columns using ColumnTransformer
numerical_columns = features.columns
ct = ColumnTransformer([('normalize', Normalizer(), numerical_columns)], remainder='passthrough')

features_train_norm = ct.fit_transform(features_train)
features_test_norm = ct.transform(features_test)

features_train_norm = pd.DataFrame(features_train_norm, columns = features_train.columns)
features_test_norm = pd.DataFrame(features_test_norm, columns = features_test.columns)

### Building the Model

In [126]:
model = Sequential()

# Creating an input Layer:
input = InputLayer(input_shape = (features.shape[1], ))

# Adding the input layer to the model:
model.add(input)

# Adding a hidden layer to the model:
model.add(Dense(16, activation = 'relu'))
model.add(layers.Dropout(0.1))

# Adding an output layer to the model:
model.add(Dense(1))

In [127]:
# Printing the summary of the model:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 16)                128       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 17        
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


### Initializing the optimizer and compiling the model:

In [133]:
opt = Adam(learning_rate = 0.005)
model.compile(loss = 'mse', metrics = ['mae'], optimizer = opt)

### Fit and evaluate the model

In [134]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
#model.fit(features_train_norm, labels_train, epochs = 50, batch_size = 1, verbose = 1)
history = model.fit(features_train_norm, labels_train, epochs=100, batch_size= 2, verbose=1, validation_split = 0.2, callbacks = [es])

Epoch 1/100
160/160 [==============================] - 0s 3ms/step - loss: 0.0096 - mae: 0.0757 - val_loss: 0.0110 - val_mae: 0.0832
Epoch 2/100
160/160 [==============================] - 0s 1ms/step - loss: 0.0104 - mae: 0.0780 - val_loss: 0.0105 - val_mae: 0.0822
Epoch 3/100
160/160 [==============================] - 0s 2ms/step - loss: 0.0092 - mae: 0.0743 - val_loss: 0.0098 - val_mae: 0.0803
Epoch 4/100
160/160 [==============================] - 0s 2ms/step - loss: 0.0107 - mae: 0.0795 - val_loss: 0.0130 - val_mae: 0.1002
Epoch 5/100
160/160 [==============================] - 0s 1ms/step - loss: 0.0102 - mae: 0.0780 - val_loss: 0.0099 - val_mae: 0.0808
Epoch 6/100
160/160 [==============================] - 0s 2ms/step - loss: 0.0102 - mae: 0.0782 - val_loss: 0.0115 - val_mae: 0.0800
Epoch 7/100
160/160 [==============================] - 0s 2ms/step - loss: 0.0111 - mae: 0.0824 - val_loss: 0.0120 - val_mae: 0.0949
Epoch 8/100
160/160 [==============================] - 0s 1ms/step - 

In [135]:
# Evaluating your model:
res_mse, res_mae = model.evaluate(features_test_norm, labels_test)
print(res_mse, res_mae)

4/4 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0626
0.00712757371366024 0.06262410432100296


In [136]:
# evauate r-squared score
y_pred = model.predict(features_test_norm)
print(r2_score(labels_test,y_pred))

0.6147807494459377
